In [1]:
!pip install fastai2

     |████████████████████████████████| 184 kB 2.7 MB/s 


In [2]:
from fastai2.vision.all import *

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
base = Path('../input/rsna-hemorrhage-jpg')
trn_path = base/'train_jpg/train_jpg'
csv_path = base/'meta/meta'

In [5]:
fn = get_image_files(trn_path)
len(fn)

194082

In [6]:
labels = pd.read_feather(csv_path/'labels.fth')
labels.head()

,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0,0,0,0,0,0,0
1,ID_00005679d,0,0,0,0,0,0
2,ID_00008ce3c,0,0,0,0,0,0
3,ID_0000950d7,0,0,0,0,0,0
4,ID_0000aee4b,0,0,0,0,0,0


In [7]:
labels.shape, fn[0]

((674258, 7),
 Path('../input/rsna-hemorrhage-jpg/train_jpg/train_jpg/ID_2369d2208.jpg'))

In [8]:
fn[0].name

'ID_2369d2208.jpg'

Extracting just the file names from the list of paths :

In [9]:
names = fn.attrgot('name')
len(names),names[0]

(194082, 'ID_2369d2208.jpg')

In [10]:
labels['ID'] = labels['ID'].astype(str)+'.jpg'
labels.head()

,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0.jpg,0,0,0,0,0,0
1,ID_00005679d.jpg,0,0,0,0,0,0
2,ID_00008ce3c.jpg,0,0,0,0,0,0
3,ID_0000950d7.jpg,0,0,0,0,0,0
4,ID_0000aee4b.jpg,0,0,0,0,0,0


In [11]:
labels = labels[labels['ID'].isin(names)]

labels.shape

(194082, 7)

In [12]:
dftrain = pd.read_feather(csv_path/'df_trn.fth')
dftrain.head().T

,0,1,2,3,4
BitsAllocated,16,16,16,16,16
BitsStored,16,12,16,16,12
Columns,512,512,512,512,512
HighBit,15,11,15,15,11
ImageOrientationPatient,1,1,1,1,1
ImageOrientationPatient1,0,0,0,0,0
ImageOrientationPatient2,0,0,0,0,0
ImageOrientationPatient3,0,0,0,0,0
ImageOrientationPatient4,0.927184,1,1,0.927184,1
ImageOrientationPatient5,-0.374607,0,0,-0.374607,0


The way names of files are stored in the labels and train data is different. One of them is saved with the extension ".jpg" hence, we add the extension to the other df in order to make the column identical to uniquely identify the data samples.
Also, we need to keep data samples which are common to both the dataframes so that we have labels for all the data samples. Hence, we merge the dfs on the ID column

In [13]:
print(dftrain.shape)
dftrain['SOPInstanceUID'] = dftrain['SOPInstanceUID'].astype(str)+'.jpg'
dftrain = dftrain[dftrain['SOPInstanceUID'].isin(names)]
dftrain.shape

(674258, 42)


(194082, 42)

In [14]:
dftrain.isnull().sum()

BitsAllocated                        0
BitsStored                           0
Columns                              0
HighBit                              0
ImageOrientationPatient              0
ImageOrientationPatient1             0
ImageOrientationPatient2             0
ImageOrientationPatient3             0
ImageOrientationPatient4             0
ImageOrientationPatient5             0
ImagePositionPatient                 0
ImagePositionPatient1                0
ImagePositionPatient2                0
Modality                             0
MultiImageOrientationPatient         0
MultiImagePositionPatient            0
MultiPixelSpacing                    0
MultiWindowCenter               103205
MultiWindowWidth                103205
PatientID                            0
PhotometricInterpretation            0
PixelRepresentation                  0
PixelSpacing                         0
PixelSpacing1                        0
RescaleIntercept                     0
RescaleSlope             

In [15]:
dftrain['Modality'].value_counts()


CT    194082
Name: Modality, dtype: int64

In [16]:
dftrain = dftrain.drop(['MultiWindowCenter','MultiWindowWidth','WindowCenter1','WindowWidth1','Modality','StudyID','PatientID','SOPInstanceUID','SeriesInstanceUID','PhotometricInterpretation','StudyInstanceUID','fname'],axis=1)

In [17]:
dftrain.isnull().sum()

BitsAllocated                   0
BitsStored                      0
Columns                         0
HighBit                         0
ImageOrientationPatient         0
ImageOrientationPatient1        0
ImageOrientationPatient2        0
ImageOrientationPatient3        0
ImageOrientationPatient4        0
ImageOrientationPatient5        0
ImagePositionPatient            0
ImagePositionPatient1           0
ImagePositionPatient2           0
MultiImageOrientationPatient    0
MultiImagePositionPatient       0
MultiPixelSpacing               0
PixelRepresentation             0
PixelSpacing                    0
PixelSpacing1                   0
RescaleIntercept                0
RescaleSlope                    0
Rows                            0
SamplesPerPixel                 0
WindowCenter                    0
WindowWidth                     0
img_max                         0
img_mean                        0
img_min                         0
img_pct_window                  0
img_std       

In [18]:
dftrain.dtypes

BitsAllocated                     int64
BitsStored                        int64
Columns                           int64
HighBit                           int64
ImageOrientationPatient         float64
ImageOrientationPatient1        float64
ImageOrientationPatient2        float64
ImageOrientationPatient3        float64
ImageOrientationPatient4        float64
ImageOrientationPatient5        float64
ImagePositionPatient            float64
ImagePositionPatient1           float64
ImagePositionPatient2           float64
MultiImageOrientationPatient      int64
MultiImagePositionPatient         int64
MultiPixelSpacing                 int64
PixelRepresentation               int64
PixelSpacing                    float64
PixelSpacing1                   float64
RescaleIntercept                float64
RescaleSlope                    float64
Rows                              int64
SamplesPerPixel                   int64
WindowCenter                    float64
WindowWidth                     float64


In [19]:
dftrain.shape

(194082, 30)

In [20]:
split = int(dftrain.shape[0]*0.8)
split

155265

In [21]:
X_train,X_valid=dftrain[:split],dftrain[split:]
Y_train,Y_valid=labels[:split],labels[split:]

In [22]:
Y_train = Y_train['any']
Y_valid = Y_valid['any']


In [23]:
def randomfor(n_estimators = 200, max_features='auto', min_samples_leaf = 1):

    rf = RandomForestClassifier(n_estimators = n_estimators,max_features =max_features, min_samples_leaf = min_samples_leaf)
    rf.fit(X_train, Y_train)
    score = rf.score(X_valid, Y_valid)
    print(score * 100)

In [24]:
randomfor()

50.23056908055749


The medadata by itself is not sufficient to achieve good results even after using a robust model like random forest. Hence, we won't focus on the medadata results and instead focus on the CNN to classify images directly.